In [1]:
import numpy as np

# Binary numbers

## 1. Practice writing numbers in binary format, i.e. in base-2, i.e. as a bit string.
1) Write the numbers from 0₁₀ to 15₁₀ in binary.

| Decimal |   0  |   1  |   2  |   3  |   4  |   5  |   6  |   7  |   8  |   9  |  10  |  11  |  12  |  13  |  14  |  15  |
|  ---:   | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |  
|  Binary | 0000 | 0001 | 0010 | 0011 | 0100 | 0101 | 0110 | 0111 | 1000 | 1001 | 1010 | 1011 | 1100 | 1101 | 1110 | 1111 |


2) Convert these binary numbers to decimal numbers:
11010₂ , 1010101₂ , 10000000000₂

|  Binary |   110110  |   1010101  |   10000000000  |
|  ---:   | :-------: | :--------: | :-------------: |
| Decimal | 32+16+4+2 |  64+16+4+1 |            1024 |
|         |        54 |  85        |            1024 |

3) Convert these decimal numbers to binary numbers:
77₁₀ , 511₁₀ , 513₁₀

| Decimal |        77 |  511                        |         513 |
|  ---:   | :-------: | :------------------------:  | :---------: |
| Decimal | 64+8+4+1  |  256+128+64+32+16+8+4+2+1   | 521+1       |
|  Binary |   1001101  |   111111111                |  1000000001 |

4) How many bits (binary digits) are needed to represent these numbers?
30₁₀ , 300₁₀ , 3000₁₀ , 30000₁₀

|  Decimal |   30      |  300        |    3000    | 30000  |
|  ----:   | :-------: | :--------:  | :--------: | :----: |
| num bits |   5       | 9           |  12        | 15     |

$N_\mathrm{bits} = \lfloor\log_2(x) + 1\rfloor$

In [2]:
# 0b110110, 0b1010101, 0b10000000000
# 0b1001101, 0b111111111, 0b1000000001

### 2. Binary addition. Calculate the following

1. 1101₂ + 0010₂ = 1111₂
2. 1101₂ + 0011₂ = 10000₂
3. 111111₂ + 000100₂ = 1000011₂

### 3. The bitwise addition modulo 2 or XOR (exlusive-OR) with the symbol can also be considered as addition without carry or as bit value toggling with a mask.

1. 1101₂ ⊕ 10₂ = 1111
2. 1101₂ ⊕ 11₂ = 1110
3. 101101001₂ ⊕ 111001011₂ = 010100010


## 4. Division by factors of 2.
1. 100₂ / 10₂ = 010
2. 1100₂ / 10₂ = ?
3. 1000₂ / 1000₂ = ?


In [3]:
0x9f

159

In [170]:
import qutip as qt
from utils import prettyprint_qutip, prettyprint


In [47]:
zero = qt.basis(2, 0)
one = qt.basis(2, 1)

zz = qt.tensor(zero, zero)
zo = qt.tensor(zero, one)
oz = qt.tensor(one, zero)
oo = qt.tensor(one, one)


psi = 1/2*zz - 1j/2*oz + 1/np.sqrt(2)*oo
print("psi =", psi.basis_expansion())
psi_1 = - 1j/2*oz + 1/np.sqrt(2)*oo
psi_1.unit().basis_expansion(), 1/np.sqrt(3), np.sqrt(2/3)

psi = (0.70710678118655+0j) |11> + -0.5j |10> + (0.5+0j) |00>


('(0.81649658092773+0j) |11> + -0.57735026918963j |10>',
 np.float64(0.5773502691896258),
 np.float64(0.816496580927726))

In [77]:
N = 2
f"{3:0{N}}", np.binary_repr(3)

('03', '11')

In [259]:
def partial_measurement(state, measurement, verbose=False):
    """Compute partial measurement for an input state, say |001>

    Parameters
    ----------
    state : qutip.Qobj
        The input state to take partial measurement of
    measurement : tuple
        Tuple of (index, value). The index is 'which' qubit to measure, and the value is 'what
         the qubit is. E.g.:
         |A> = 1/np.sqrt(2)*|10> - 1/np.sqrt(2)|11>,
         the first term has value 0 for index 1 and both qubits have value 1 for index 0.

    Returns
    -------
    qutip.Qobj
        The output for a partial measurement ('collapsing' onto a subset of the state).
    """
    
    assert isinstance(measurement, tuple) and len(measurement) == 2, \
        "Must be a tuple of 'which' and 'value' measurement."
    which, value = measurement
    assert value in (0,1), "Measurement value must be 0 or 1"
    
    N, _ = state.shape # number of state dimensions
    num_qubits = int(np.log2(N))
    assert 2**num_qubits == N, "State dimension is not a power of 2"
    
    amps = np.asarray(state.full()).squeeze()
    bitstrings = [np.binary_repr(i, num_qubits) for i in range(N)]
    
    # Select indicies mathcing measurement 'which'
    selected_indices = [i for i, b in enumerate(bitstrings) if b[which] == str(value)]
    
    prob = np.sum(np.abs(amps[selected_indices])**2)
    
    remaining_qubits = num_qubits - 1
    new_dim = 2**remaining_qubits 
    collapsed = np.zeros(shape=(new_dim, 1), dtype=complex)
    
    for i, b in enumerate(bitstrings):
        if b[which] == str(value):
            reduced_bit = b[:which] + b[which+1:]
            reduced_idx = int(reduced_bit, 2) # convert to base-10 integer to index new 'collapsed' state
            collapsed[reduced_idx] += amps[i]
            
    collapsed = qt.Qobj(collapsed.reshape(-1,1)) # make it a Qobj 
    if prob > 0:
        collapsed /= np.sqrt(prob)
    else:
        print(f"Input state : {state.basis_expansion()}")
        print(f"Measurement : {which} → {value} is zero!")
        return None
    
    if verbose:
        print(f"Input state : {state.basis_expansion()}")
        print(f"Measurement : index {which} → {value}")
        print(f"Probability : {prob:.2f}")
        print(f"Post-measurement state : {collapsed.basis_expansion()}")
    
    return collapsed
    
    




partial_measurement(psi, (0, 1), verbose=True)
    
    

Input state : (0.70710678118655+0j) |11> + -0.5j |10> + (0.5+0j) |00>
Measurement : index 0 → 1
Probability : 0.75
Post-measurement state : (0.81649658092773+0j) |1> + -0.57735026918963j |0>


Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[0.        -0.57735027j]
 [0.81649658+0.j        ]]

In [229]:

bitstring = "1001"
which = 2
reduced_bit = bitstring[:which] + bitstring[which+1:]
print(reduced_bit, type(reduced_bit))
int(reduced_bit, 2)
# int(, 2)

101 <class 'str'>


5

In [164]:
psi.dims, "", one.data

([[2, 2], [1]], '', Dense(shape=(2, 1), fortran=True))

In [129]:
"01"[1] == str("1")

True

In [272]:
bell_gen = qt.tensor(qt.gates.hadamard_transform(1), qt.identity(2))*qt.gates.cnot()

display(bell_gen)
inv_bell_gen = qt.gates.cnot()* qt.tensor(qt.gates.hadamard_transform(1), qt.identity(2))
display(inv_bell_gen)

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=False
Qobj data =
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.70710678  0.          0.         -0.70710678]
 [ 0.          0.70710678 -0.70710678  0.        ]]

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=False
Qobj data =
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.          0.70710678  0.         -0.70710678]
 [ 0.70710678  0.         -0.70710678  0.        ]]

In [ ]:
psi = (qt.basis(2, 0) + qt.basis(2,1)).unit()

inp = qt.tensor(psi, qt.basis(2,0), qt.basis(2,0))

bell_gen = qt.tensor(qt.identity(2), qt.gates.cnot())*qt.tensor(qt.identity(2), qt.gates.hadamard_transform(1), qt.identity(2))
inv_bell_gen = qt.tensor(qt.gates.cnot(), qt.identity(2)) * qt.tensor(qt.gates.hadamard_transform(1), qt.identity(2), qt.identity(2))

partial_measurement(inv_bell_gen*bell_gen*inp, (0, 0), verbose=True)

# partial_measurement(inv_bell_gen*bell_gen*inp, (0, 1))

Input state : (0.70710678118655+0j) |000> + (0.70710678118655+0j) |011>
Measurement : index 0 → 0
Probability : 1.00
Post-measurement state : (0.70710678118655+0j) |3> + (0.70710678118655+0j) |0>
Input state : (0.70710678118655+0j) |000> + (0.70710678118655+0j) |011>
Measurement : index 1 → 0
Probability : 0.50
Post-measurement state : (1+0j) |0>


Quantum object: dims=[[4], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]]